In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

import json
import os

In [ ]:
## 설정
BATCH_SIZE= 256
EPOCHS = 100
LEARNING_RATE = 0.01
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = torchvision.datasets.ImageNet('./data/imagenet', split='train', download=None, transform=transform)
train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.ImageNet('./data/imagenet', split='val', download=None, transform=transform)
test_dataloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [9]:
class VGG16Net(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(VGG16Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(64, 64, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(128, 128, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(128, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(256, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2)
        )
        
        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=4096, out_features=num_classes),
        )

    def forward(self, x):
        features = self.conv(x)
        # x = self.avgpool(x)
        x = x.view(features.size(0), -1)
        x = self.fcs(x)
        return x

In [12]:
model = VGG16Net(10).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(lr=LEARNING_RATE, weight_decay=5e-3, params=model.parameters(), momentum=0.9)


In [ ]:
for epoch in range(EPOCHS):
    for idx, _data in enumerate(train_dataloader, start=0):
        